In [19]:
import import_ipynb
import sudoku_solver as ss
import img_preprocessing as ip
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [1]:
# Read in the image and the model 
pathImage = ""
heightImg = 450
widthImg = 450
model = ip.intializePredectionModel() 

# Preprocess and fit the image to size
img = cv2.imread(pathImage)
img = cv2.resize(img, (widthImg, heightImg))
imgBlank = np.zeros((heightImg, widthImg, 3), np.uint8) # Test
imgThreshold = preProcess(img)

# Finding image contours
imgContours = img.copy() # Test
imgBigContour = img.copy() # Test
contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Find contours, external for board
cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 3) # Display the contours found

# Find largest contour 
biggest, maxArea = biggestContour(contours)

if biggest.size != 0:
    biggest = reorder(biggest)
    print(biggest)
    cv2.drawContours(imgBigContour, biggest, -1, (255, 0, 0), 30)
    pts1 = np.float32(biggest) # Points for warping
    pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # Points for warping
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
    imgDetectedDigits = imgBlank.copy()
    imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)

    # Splitting image into boxes and getting the digit in every box
    imgSolvedDigits = imgBlank.copy()
    boxes = splitBoxes(imgWarpColored)
    numbers = getPredection(boxes, model)
    imgDetectedDigits = displayNumbers(imgDetectedDigits, numbers, color=(255, 0, 255))
    numbers = np.asarray(numbers)
    posArray = np.where(numbers > 0, 0, 1) # prints value 1 where needs to find digit


    # Finds the solution to the board
    board = np.array_split(numbers,9)
    print(board)
    try:
        ss.solve(board)
    except:
        pass
    print(board)
    flatList = []
    for sublist in board:
        for item in sublist:
            flatList.append(item)
    solvedNumbers =flatList*posArray
    imgSolvedDigits= displayNumbers(imgSolvedDigits,solvedNumbers)

    # Prints the solution to the board
    pts2 = np.float32(biggest) # Points for warping
    pts1 =  np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # Points for warping
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgInvWarpColored = img.copy()
    imgInvWarpColored = cv2.warpPerspective(imgSolvedDigits, matrix, (widthImg, heightImg))
    inv_perspective = cv2.addWeighted(imgInvWarpColored, 1, img, 0.5, 1)
    imgDetectedDigits = drawGrid(imgDetectedDigits)
    imgSolvedDigits = drawGrid(imgSolvedDigits)

imageArray = ([img,imgThreshold,imgContours, imgBigContour],
              [imgDetectedDigits, imgSolvedDigits,imgInvWarpColored,inv_perspective])

stackedImage = stackImages(imageArray, 1)
cv2_imshow(stackedImage)

else:
    print("Not a valid board")
